In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report

In [4]:
df=pd.read_csv("Airline_review.csv")
df.head(3)

,Unnamed: 0,Airline Name,Overall_Rating,Review_Title,Review Date,Verified,Review,Aircraft,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Wifi & Connectivity,Value For Money,Recommended
0,0,AB Aviation,9,"""pretty decent airline""",11th November 2019,True,Moroni to Moheli. Turned out to be a pretty ...,NaN,Solo Leisure,Economy Class,Moroni to Moheli,November 2019,4.0,5.0,4.0,4.0,NaN,NaN,3.0,yes
1,1,AB Aviation,1,"""Not a good airline""",25th June 2019,True,Moroni to Anjouan. It is a very small airline...,E120,Solo Leisure,Economy Class,Moroni to Anjouan,June 2019,2.0,2.0,1.0,1.0,NaN,NaN,2.0,no
2,2,AB Aviation,1,"""flight was fortunately short""",25th June 2019,True,Anjouan to Dzaoudzi. A very small airline an...,Embraer E120,Solo Leisure,Economy Class,Anjouan to Dzaoudzi,June 2019,2.0,1.0,1.0,1.0,NaN,NaN,2.0,no


In [5]:
df.drop("Unnamed: 0",axis=1,inplace=True)
df.drop("Date Flown",axis=1,inplace=True)

In [7]:
# Count occurrences of 'recommended' and 'not recommended'
yes = df['Recommended'].value_counts()['yes']
no = df['Recommended'].value_counts()['no']

print(f"Number of 'recommended': {yes}")
print(f"Number of 'not recommended': {no}")


Number of 'recommended': 7807
Number of 'not recommended': 15364


In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23171 entries, 0 to 23170
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Airline Name            23171 non-null  object 
 1   Overall_Rating          23171 non-null  object 
 2   Review_Title            23171 non-null  object 
 3   Review Date             23171 non-null  object 
 4   Verified                23171 non-null  bool   
 5   Review                  23171 non-null  object 
 6   Aircraft                7129 non-null   object 
 7   Type Of Traveller       19433 non-null  object 
 8   Seat Type               22075 non-null  object 
 9   Route                   19343 non-null  object 
 10  Seat Comfort            19016 non-null  float64
 11  Cabin Staff Service     18911 non-null  float64
 12  Food & Beverages        14500 non-null  float64
 13  Ground Service          18378 non-null  float64
 14  Inflight Entertainment  10829 non-null

In [70]:
# Checking duplicate rows:
df.duplicated().value_counts()

False    23051
True       120
dtype: int64

In [71]:
df = df.drop_duplicates().reset_index(drop=True)

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23051 entries, 0 to 23050
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Airline Name            23051 non-null  object 
 1   Overall_Rating          23051 non-null  object 
 2   Review_Title            23051 non-null  object 
 3   Review Date             23051 non-null  object 
 4   Verified                23051 non-null  bool   
 5   Review                  23051 non-null  object 
 6   Aircraft                7127 non-null   object 
 7   Type Of Traveller       19424 non-null  object 
 8   Seat Type               22062 non-null  object 
 9   Route                   19335 non-null  object 
 10  Seat Comfort            19007 non-null  float64
 11  Cabin Staff Service     18902 non-null  float64
 12  Food & Beverages        14496 non-null  float64
 13  Ground Service          18369 non-null  float64
 14  Inflight Entertainment  10827 non-null

In [73]:
df.duplicated().value_counts()

False    23051
dtype: int64

In [74]:
# For categorical columns
df['Aircraft']=df['Aircraft'].fillna(df['Aircraft'].mode().iloc[0])
df['Type Of Traveller']=df['Type Of Traveller'].fillna(df['Type Of Traveller'].mode().iloc[0])
df['Seat Type']=df['Seat Type'].fillna(df['Seat Type'].mode().iloc[0])
df['Route']=df['Route'].fillna(df['Route'].mode().iloc[0])

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23051 entries, 0 to 23050
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Airline Name            23051 non-null  object 
 1   Overall_Rating          23051 non-null  object 
 2   Review_Title            23051 non-null  object 
 3   Review Date             23051 non-null  object 
 4   Verified                23051 non-null  bool   
 5   Review                  23051 non-null  object 
 6   Aircraft                23051 non-null  object 
 7   Type Of Traveller       23051 non-null  object 
 8   Seat Type               23051 non-null  object 
 9   Route                   23051 non-null  object 
 10  Seat Comfort            19007 non-null  float64
 11  Cabin Staff Service     18902 non-null  float64
 12  Food & Beverages        14496 non-null  float64
 13  Ground Service          18369 non-null  float64
 14  Inflight Entertainment  10827 non-null

In [76]:
# For numerical columns
l=['Seat Comfort','Cabin Staff Service','Food & Beverages','Ground Service','Inflight Entertainment','Wifi & Connectivity','Value For Money']
for i in l:
    df[i]=df[i].fillna(df[i].median())

## EDA (Data Visualization):

In [77]:
## Removing unnecessary columns:
df.drop("Route",axis=1,inplace=True)
df.drop("Aircraft",axis=1,inplace=True)
df.drop("Review Date",axis=1,inplace=True)
df.drop("Airline Name",axis=1,inplace=True)


## Encoding

In [78]:
df['Verified']=df['Verified'].replace({'True':1,'False':0})
df['Type Of Traveller']=df['Type Of Traveller'].replace({'Solo Leisure':1,'Couple Leisure':2,'Family Leisure':3,'Business':4})
df['Seat Type']=df['Seat Type'].replace({'Economy Class':1,'Business Class':2,'Premium Economy':3,'First Class':4})
df['Recommended']=df['Recommended'].replace({'yes':1,'no':0})

df['Overall_Rating']=df['Overall_Rating'].replace('n',10)

In [79]:
df['Overall_Rating']=df['Overall_Rating'].astype(int)

In [80]:
l=["Overall_Rating","Review_Title","Cabin Staff Service","Food & Beverages","Recommended"]
df=df[l]

df.head(3)

,Overall_Rating,Review_Title,Cabin Staff Service,Food & Beverages,Recommended
0,9,"""pretty decent airline""",5.0,4.0,1
1,1,"""Not a good airline""",2.0,1.0,0
2,1,"""flight was fortunately short""",1.0,1.0,0


## Preprocessing using spacy:

In [81]:
!pip install spacy
!python -m spacy download en_core_web_sm


                                              0.0/12.8 MB ? eta -:--:--
                                              0.1/12.8 MB 2.6 MB/s eta 0:00:05
                                              0.1/12.8 MB 1.2 MB/s eta 0:00:11
                                              0.2/12.8 MB 1.4 MB/s eta 0:00:10
                                              0.3/12.8 MB 1.3 MB/s eta 0:00:10
                                              0.3/12.8 MB 1.6 MB/s eta 0:00:08
     -                                        0.4/12.8 MB 1.5 MB/s eta 0:00:09
     -                                        0.5/12.8 MB 1.5 MB/s eta 0:00:09
     -                                        0.5/12.8 MB 1.5 MB/s eta 0:00:09
     -                                        0.6/12.8 MB 1.4 MB/s eta 0:00:09
     -                                        0.6/12.8 MB 1.4 MB/s eta 0:00:09
     --                                       0.7/12.8 MB 1.3 MB/s eta 0:00:10
     --                                       0.7/12.8 MB 1

In [82]:
nlp=spacy.load("en_core_web_sm")

In [83]:
# Important words (so that they are not lost)
exceptions = ["not", "never", "bad", "nice", "good", "great", "poor", "excellent","no"]
def preprocess (text):
    text = text.lower() # to convert to lowercase (so that we can add exceptions properly)
    doc=nlp(text)
    filtered_tokens=[]
    for token in doc:
        if token.text in exceptions:
            filtered_tokens.append(token.text)
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    return " ".join(filtered_tokens)

In [84]:
df['Review']=df['Review_Title'].apply(lambda x: preprocess(x))

In [85]:
df.head(5)

,Overall_Rating,Review_Title,Cabin Staff Service,Food & Beverages,Recommended,Review
0,9,"""pretty decent airline""",5.0,4.0,1,pretty decent airline
1,1,"""Not a good airline""",2.0,1.0,0,not good good airline
2,1,"""flight was fortunately short""",1.0,1.0,0,flight fortunately short
3,1,"""I will never fly again with Adria""",1.0,2.0,0,never fly adria
4,1,"""it ruined our last days of holidays""",1.0,1.0,0,ruin day holiday


In [86]:
df.columns

Index(['Overall_Rating', 'Review_Title', 'Cabin Staff Service',
       'Food & Beverages', 'Recommended', 'Review'],
      dtype='object')

In [87]:
df.head()

,Overall_Rating,Review_Title,Cabin Staff Service,Food & Beverages,Recommended,Review
0,9,"""pretty decent airline""",5.0,4.0,1,pretty decent airline
1,1,"""Not a good airline""",2.0,1.0,0,not good good airline
2,1,"""flight was fortunately short""",1.0,1.0,0,flight fortunately short
3,1,"""I will never fly again with Adria""",1.0,2.0,0,never fly adria
4,1,"""it ruined our last days of holidays""",1.0,1.0,0,ruin day holiday


## Text Vectorization

In [88]:
v1=TfidfVectorizer()
v1.fit(df["Review"])
rev_tfidf = v1.transform(df["Review"])

In [89]:
rev_tfidf

<23051x3892 sparse matrix of type '<class 'numpy.float64'>'
	with 68754 stored elements in Compressed Sparse Row format>

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23051 entries, 0 to 23050
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Overall_Rating       23051 non-null  int32  
 1   Review_Title         23051 non-null  object 
 2   Cabin Staff Service  23051 non-null  float64
 3   Food & Beverages     23051 non-null  float64
 4   Recommended          23051 non-null  int64  
 5   Review               23051 non-null  object 
dtypes: float64(2), int32(1), int64(1), object(2)
memory usage: 990.6+ KB


## Splitting

In [91]:
# Combine TF-IDF vectors with the existing numerical columns:

# pd.DataFrame -- to convert into a dataframe
# .toarray() -- converts the sparse TF-IDF matrix into a dense matrix
# pd.concat([...], axis=1) -- concatenates the DataFrames created in the first two parts (TF-IDF vectors for "rev" and "revTitle") and the DataFrame from the third part (remaining numerical columns)

x = pd.concat([pd.DataFrame(rev_tfidf.toarray()), 
               df.drop(columns=["Review_Title", "Review","Recommended"])], axis=1)

In [92]:
y=df["Recommended"]

In [93]:
x

,0,1,2,3,4,5,6,7,8,9,...,3885,3886,3887,3888,3889,3890,3891,Overall_Rating,Cabin Staff Service,Food & Beverages
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,5.0,4.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,2.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23046,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,2.0
23047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3.0,2.0
23048,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,4.0,2.0
23049,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,4.0,3.0


In [94]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23051 entries, 0 to 23050
Columns: 3895 entries, 0 to Food & Beverages
dtypes: float64(3894), int32(1)
memory usage: 684.9 MB


In [95]:
y.info()

<class 'pandas.core.series.Series'>
RangeIndex: 23051 entries, 0 to 23050
Series name: Recommended
Non-Null Count  Dtype
--------------  -----
23051 non-null  int64
dtypes: int64(1)
memory usage: 180.2 KB


In [96]:
x.columns = x.columns.astype(str)

In [97]:
# scaler = MinMaxScaler()
# scaler.fit(x)
# x = scaler.transform(x)
# x = pd.DataFrame(x)
# x.info()

## TrainTestSplit

In [98]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [99]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(18440, 3895)
(4611, 3895)
(18440,)
(4611,)


In [100]:
y_train

14149    0
3728     1
16249    1
450      0
17987    1
        ..
11964    0
21575    0
5390     0
860      0
15795    0
Name: Recommended, Length: 18440, dtype: int64

## Model Training

## 1) Neural Networks

In [101]:
model1 = keras.Sequential([
    keras.layers.Input(shape=(x_train.shape[1],)),  # shape of the input data
    keras.layers.Dense(64, activation='relu'),  # hidden layer
    keras.layers.Dense(1, activation='sigmoid')
])

model1.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the model
model1.fit(x_train, y_train, epochs=50)

Epoch 1/50
577/577 [==============================] - 13s 6ms/step - loss: 0.3160 - accuracy: 0.8858
Epoch 2/50
577/577 [==============================] - 3s 5ms/step - loss: 0.1768 - accuracy: 0.9308
Epoch 3/50
577/577 [==============================] - 3s 5ms/step - loss: 0.1497 - accuracy: 0.9422
Epoch 4/50
577/577 [==============================] - 3s 5ms/step - loss: 0.1328 - accuracy: 0.9499
Epoch 5/50
577/577 [==============================] - 3s 5ms/step - loss: 0.1214 - accuracy: 0.9544
Epoch 6/50
577/577 [==============================] - 3s 5ms/step - loss: 0.1118 - accuracy: 0.9583
Epoch 7/50
577/577 [==============================] - 3s 5ms/step - loss: 0.1057 - accuracy: 0.9624
Epoch 8/50
577/577 [==============================] - 3s 5ms/step - loss: 0.0997 - accuracy: 0.9632
Epoch 9/50
577/577 [==============================] - 3s 5ms/step - loss: 0.0937 - accuracy: 0.9672
Epoch 10/50
577/577 [==============================] - 3s 6ms/step - loss: 0.0902 - accuracy: 0.969

In [102]:
# Evaluate the model
loss, accuracy = model1.evaluate(x_train, y_train)
print(f"Test accuracy: {accuracy}")

577/577 [==============================] - 3s 3ms/step - loss: 0.0374 - accuracy: 0.9880
Test accuracy: 0.9880151748657227


## 2) Random Forest Classifier

In [103]:
model2=RandomForestClassifier(n_estimators=100, random_state=42)
model2

RandomForestClassifier(random_state=42)

In [104]:
model2.fit(x_train, y_train)
y_pred2 = model2.predict(x_test)
print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

           0       0.95      0.96      0.96      3051
           1       0.92      0.90      0.91      1560

    accuracy                           0.94      4611
   macro avg       0.94      0.93      0.93      4611
weighted avg       0.94      0.94      0.94      4611



## 3) KNeighbors Classifier

In [105]:
model3=KNeighborsClassifier()
model3

KNeighborsClassifier()

In [106]:
model3.fit(x_train, y_train)
y_pred3 = model3.predict(x_test)
print(classification_report(y_test, y_pred3))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95      3051
           1       0.91      0.90      0.91      1560

    accuracy                           0.94      4611
   macro avg       0.93      0.93      0.93      4611
weighted avg       0.94      0.94      0.94      4611



## 4) Decision Tree Classifier

In [107]:
model4=DecisionTreeClassifier() # Increase the max_iter value
model4

DecisionTreeClassifier()

In [108]:
model4.fit(x_train, y_train)
y_pred4 = model4.predict(x_test)
print(classification_report(y_test, y_pred4))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95      3051
           1       0.90      0.90      0.90      1560

    accuracy                           0.93      4611
   macro avg       0.92      0.92      0.92      4611
weighted avg       0.93      0.93      0.93      4611



## 5) Logistic Regression

In [109]:
model5=LogisticRegression(max_iter=1000) # Increase the max_iter value
model5

LogisticRegression(max_iter=1000)

In [110]:
model5.fit(x_train, y_train)
y_pred5 = model5.predict(x_test)
print(classification_report(y_test, y_pred5))
print(confusion_matrix(y_test,y_pred5))

              precision    recall  f1-score   support

           0       0.92      0.93      0.92      3051
           1       0.85      0.84      0.85      1560

    accuracy                           0.90      4611
   macro avg       0.89      0.88      0.88      4611
weighted avg       0.90      0.90      0.90      4611

[[2827  224]
 [ 254 1306]]


In [111]:
# Sample input data
input_data = [['This airline is great', 9, 5, 4], ['Not a good experience', 2, 3, 2]]

# Create a DataFrame with the input data
input_df = pd.DataFrame(input_data, columns=['Review', 'Overall_Rating', 'Cabin Staff Service', 'Food & Beverages'])

# Preprocess the input data
# 1. Use the same TfidfVectorizer that you used for training data
rev_tfidf = v1.transform(input_df["Review"])

# 2. Concatenate the text features and numerical features
x_input = pd.concat([pd.DataFrame(rev_tfidf.toarray()), input_df.drop(columns=["Review"])], axis=1)
x_input.columns = x_input.columns.astype(str)

# Predict "Recommended" for the input data
y_pred_input = model1.predict(x_input)

# Selecting 0.5 as a threshold in model1
for i, review in enumerate(input_data):
    if(y_pred_input[i]<=0.5):    
        print(f'Review: {review[0]}, Prediction: Not Recommended')
    if(y_pred_input[i]>0.5):
        print(f'Review: {review[0]}, Prediction: Recommended')
    

1/1 [==============================] - 1s 986ms/step
Review: This airline is great, Prediction: Recommended
Review: Not a good experience, Prediction: Not Recommended


In [113]:
df = pd.read_csv(r"C:\Users\CHARUTHA MURARI S\Desktop\AIRLINE REVIEW CLASSIFICATION USING ML\ml_model\Airline_review.csv")

In [114]:
model1.save(r"C:\Users\CHARUTHA MURARI S\Desktop\AIRLINE REVIEW CLASSIFICATION USING ML\ml_model\AIRLINE REVIEW CLASSIFICATION USING ML.h5")

C:\Users\CHARUTHA MURARI S\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [116]:
model1 = load_model(r"C:\Users\CHARUTHA MURARI S\Desktop\AIRLINE REVIEW CLASSIFICATION USING ML\ml_model\AIRLINE REVIEW CLASSIFICATION USING ML.h5")

In [124]:
import joblib

joblib.dump(v1,r"C:\Users\CHARUTHA MURARI S\Desktop\AIRLINE REVIEW CLASSIFICATION USING ML\ml_model\AIRLINE REVIEW CLASSIFICATION USING ML.pkl")

['C:\\Users\\CHARUTHA MURARI S\\Desktop\\AIRLINE REVIEW CLASSIFICATION USING ML\\ml_model\\AIRLINE REVIEW CLASSIFICATION USING ML.pkl']

In [125]:
from tensorflow.keras.models import load_model

model1 = load_model(r"C:\Users\CHARUTHA MURARI S\Desktop\AIRLINE REVIEW CLASSIFICATION USING ML\ml_model\AIRLINE REVIEW CLASSIFICATION USING ML.h5")

In [126]:
import joblib

v1 = joblib.load(r"C:\Users\CHARUTHA MURARI S\Desktop\AIRLINE REVIEW CLASSIFICATION USING ML\ml_model\AIRLINE REVIEW CLASSIFICATION USING ML.pkl")